# Text Summarizer

In [1]:
import summarizer
import pickle
import torch

In [2]:
from summarizer import Summarizer

In [3]:
def summary_perc(initial_text, summarized_text):
    '''
    The purpose of this function is to output a % associated to the length of the text which was reduced and summarized.
    
    args:
        initial_text (str) : This variable holds the initial body of text passed throguh the model
        summarized_text (str) : This holds the output text which was summarized by the model
    
    returns:
        A percentage associated to how much from the initial text was summarized
        
    example:
        summary_perc(initial_text = body, summarized_text = result)
    '''
    percentage = len(summarized_text) / len(initial_text)
    return print('The initial text was reduced by : ', 1 - percentage)

### Call Pre Trained Bert Model

In [4]:
model = Summarizer()

### Pass in Text

In [5]:
text = '''
Clearly, the number of pieces of information that must be found and used for bauxite to become, say, the aluminum sheeting that forms the cas- ing of the printing press that produced the pages that you are now reading is staggeringly large. It is a number far larger than the mere one billion pieces of the jigsaw puzzle in my example.
It’s foolish to expect any one person (or small group of people) to find all the pieces of information necessary for the production of aluminum sheeting (and for the production of fuselages for airliners, the production of oven foil, the production of soda cans ... the list is long).
Not only is the mere finding of all the many pieces of information too difficult to entrust to a small group of people; so, too, is the task of putting these pieces together in a way that yields useful final products.
Let’s now amend the example to make the jigsaw puzzle an even bet- ter metaphor for economic reality. Suppose that, unlike with regular jigsaw puzzles, each piece of this puzzle can be made to fit snugly and smoothly with any other piece. In this case, merely assembling all of the one billion puzzle pieces so that they fit together neatly is easy. But note that it is possible to create an unfathomably large number of scenes with these pieces.
Trouble is, only a tiny handful of these scenes will please the human eye. Most of the scenes will be visual gibberish. The challenge is to arrange the pieces together so that the final result is a recognizable scene—say, of a field of sunflowers or of a bustling city street. Only if the scene is recognizable is the assembled puzzle valuable.
Now imagine yourself standing alone before a gigantic table covered with these one billion puzzle pieces. What are the chances that you alone can put these pieces together so that the final result is a coherent visual image—a useful and valuable final result?
'''


In [6]:
result = model(text, min_length=60)

In [7]:
result

'Clearly, the number of pieces of information that must be found and used for bauxite to become, say, the aluminum sheeting that forms the cas- ing of the printing press that produced the pages that you are now reading is staggeringly large. Trouble is, only a tiny handful of these scenes will please the human eye.'

In [11]:
'The Summarized Text : {}'.format(result)

'The Summarized Text : Clearly, the number of pieces of information that must be found and used for bauxite to become, say, the aluminum sheeting that forms the cas- ing of the printing press that produced the pages that you are now reading is staggeringly large. Trouble is, only a tiny handful of these scenes will please the human eye.\n'

In [12]:
len(text)

1892

In [13]:
len(result)

315

In [14]:
summary_perc(initial_text = text, summarized_text = result)


The initial text was reduced by :  0.8335095137420718


### Save Model

In [15]:
filename = '../summarizer_model.pt'

In [16]:
torch.save(model, filename)

In [17]:
model = torch.load(filename)
# model.eval()